In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.integrate import quad
import math
import os
import sys
import copy
from tqdm import tqdm
import time

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
os.environ['PYTHONHASHSEED'] = '1234'

In [4]:
if os.environ.get("PYTHONHASHSEED") == "0":
    raise Exception("You must set PYTHONHASHSEED=0 when starting the Jupyter server to get reproducible results.")

## Data

Download and prepare.

email: http://snap.stanford.edu/data/email-Eu-core.html

In [93]:
%%bash
file_email=./data/email-Eu-core.txt.gz
file_wiki=./data/wiki-Vote.txt.gz

if [ -e "$file_email" ]; then
    echo "$file_email exists, skipping"
else 
    echo "$file_email does not exist, using wget to download it"
    wget http://snap.stanford.edu/data/email-Eu-core.txt.gz -P data/
    gzip -dkfv $file_email
fi

if [ -e "$file_wiki" ]; then
    echo "$file_wiki exists, skipping"
else 
    echo "$file_wiki does not exist, using wget to download it"
    wget http://snap.stanford.edu/data/wiki-Vote.txt.gz -P data/
    gzip -dkfv $file_wiki
fi

./data/email-Eu-core.txt.gz does not exist, using wget to download it
./data/wiki-Vote.txt.gz does not exist, using wget to download it


--2021-11-27 22:25:44--  http://snap.stanford.edu/data/email-Eu-core.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79754 (78K) [application/x-gzip]
Saving to: ‘data/email-Eu-core.txt.gz’

     0K .......... .......... .......... .......... .......... 64%  122K 0s
    50K .......... .......... .......                         100% 68,2K=0,8s

2021-11-27 22:25:45 (95,1 KB/s) - ‘data/email-Eu-core.txt.gz’ saved [79754/79754]

./data/email-Eu-core.txt.gz:	 58.6% -- created ./data/email-Eu-core.txt
--2021-11-27 22:25:45--  http://snap.stanford.edu/data/wiki-Vote.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290339 (284K) [application/x-gzip]
Saving to: ‘data/wiki-Vote.txt

In [94]:
def read_data(from_file_name, experiment):
    """ Read data. """
    
    if experiment == "email":
        df_edges = pd.read_csv(from_file_name, sep=" ", header=None, skiprows=0, names=["src", "dst"])

        vertices_np = \
            np.unique(np.hstack([df_edges["src"].unique(), df_edges["dst"].unique()]))

        df_vertices = pd.DataFrame(data={"id": vertices_np})
    else:
        df_edges = pd.read_csv(from_file_name, sep="\t", header=None, skiprows=4, names=["src", "dst"])
        vertices_np = \
            np.unique(np.hstack([df_edges["src"].unique(), df_edges["dst"].unique()]))

        df_vertices = pd.DataFrame(data={"id": vertices_np})


    return df_edges, df_vertices

In [95]:
df_edges, df_vertices = read_data(from_file_name="data/email-Eu-core.txt", experiment="email")
print(df_edges)
print(df_vertices)

g = nx.from_pandas_edgelist(
    df=df_edges, 
    source="src", 
    target="dst", 
    edge_attr=None, 
    create_using=nx.DiGraph,
    edge_key=None
)
print(g)
csv_save_path = Path("assets/wiki.csv")


       src  dst
0        0    1
1        2    3
2        2    4
3        5    6
4        5    7
...    ...  ...
25566  420  143
25567  174  859
25568  440  460
25569   52  786
25570  506  932

[25571 rows x 2 columns]
        id
0        0
1        1
2        2
3        3
4        4
...    ...
1000  1000
1001  1001
1002  1002
1003  1003
1004  1004

[1005 rows x 1 columns]
DiGraph with 1005 nodes and 25571 edges


In [18]:
df_edges, df_vertices = read_data(from_file_name="data/Wiki-Vote.txt", experiment="wiki")
print(df_edges)
print(df_vertices)

g = nx.from_pandas_edgelist(
    df=df_edges, 
    source="src", 
    target="dst", 
    edge_attr=None, 
    create_using=nx.DiGraph,
    edge_key=None
)
print(g)

         src   dst
0         30  1412
1         30  3352
2         30  5254
3         30  5543
4         30  7478
...      ...   ...
103684  8272  4940
103685  8273  4940
103686  8150  8275
103687  8150  8276
103688  8274  8275

[103689 rows x 2 columns]
        id
0        3
1        4
2        5
3        6
4        7
...    ...
7110  8293
7111  8294
7112  8295
7113  8296
7114  8297

[7115 rows x 1 columns]
DiGraph with 7115 nodes and 103689 edges


In [19]:
def plot_graph(g, path_save=None):
    plt.figure(figsize=(12,12))
    nx.draw(g)
    
    if path_save is not None:
        plt.savefig(path_save)
        
    plt.show()

In [20]:
#plot_graph(g=g, path_save=Path("assets/g_email.png"))

In [21]:
class HyperLogLogCounter():
    """ HyperLogLogCounter from https://arxiv.org/pdf/1308.2144v2.pdf
    alpha is calcualted as in http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf
    """
    def __init__(self, b, hash_to, debug=False):
        """Init.
        
        Parameters
        ----------
        b : int
            Number of bits in counter registers. Defines the number of registers p=2^b.
        hash_to : int
            Hashing to n bits. E.g.: 32 so hasing items (str or str of int) to 32 bits integers.
        debug : bool
            If True, print debug info.
        """
        # p is the number of registers in the counter, analogous to precision
        self.b = b
        self.p = 2**(self.b)
        # hash node ids to hash_to number of bits
        self.hash_to = hash_to
        self.format = f'#0{self.hash_to + 2}b'
        # general hash mask to get hash_values with hash_to number of bits
        self.hash_mask = pow(2, self.hash_to) - 1
        # mask for getting the left b number of bits as int
        self.hash_mask_left = 2**self.b - 1
        # mask for getting the hash_to - b (rest from right) number of bits as int
        self.hash_mask_right = self.hash_mask >> self.b
        # counter register (array)
        self.counter = [-np.inf for i in range(self.p)]
        # alpha from http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf
        self.alpha = (self.p * quad(lambda u: math.log2((2 + u) / (1 + u)) ** self.p, 0, np.inf)[0])**(-1)
        self.debug = debug
    
    def hash_func(self, x):
        """Hash a node id to an int. E.g.: 32-bit hash of the string value of node ID as int
        
        Parameters
        ----------
        x : int or str
            Node id.
        
        Returns
        -------
        int
            Hashed node id. Hashing to self.hash_to bits (not infinite as in paper).
        """
        xh = hash(str(x)) & self.hash_mask
        if self.debug:
            print(f"x={x} hashed to xh={format(xh, self.format)}")
        return xh
    
    def get_left_right_split(self, xh):
        """Get the split of the binary representation of a node id - left self.b bits and the rest.
        
        Parameters
        ----------
        xh : int
            Hashed node id, binary representation if self.hash_to bits long.
        
        Returns
        -------
        int, int
            left is the left self.b bits as int, and right is the rest of the bits on the right as int 
        """
        # get the left b number of bits as int
        left = xh >> (self.hash_to - self.b) & self.hash_mask_left
        # get the right hash_to-b number of bits as int
        right = xh & self.hash_mask_right
        if self.debug:
            print(f"xh={format(xh, self.format)}, left={format(left, f'#0{self.b+2}b')}, "
                  f"right={format(right, f'#0{self.hash_to-self.b+2}b')}")
        return left, right
    
    def get_leading_zeros_plus_one(self, h):
        """Get the number of leading zeroes +1 in the binary repr of a 
        hashed int (the right hand-side bits usually).
        
        Parameters
        ----------
        h : int
            Hashed value, usually righ-hand side bits, of length self.hash_to - self.b.
        
        Returns
        -------
        l0s: int
            The number of leading zeroes of the binary repr of the right hand-side bit sequence 
            that is self.hash_to - self.b long. 
        """
        l0s = self.hash_to - self.b - h.bit_length() + 1
        if self.debug:
            print(f"h={format(h, self.format)}, l0+1={l0s}")
        return l0s
    
    def size(self, ):
        """Get the size of a counter.
        
        Parameters
        ----------
        None
        
        Returns
        -------
        e: float
            The size of the counter.
        """
        z = sum([2**(-el) for el in self.counter])**(-1)
        e = self.alpha * z * self.p**2
        if self.debug:
            print(f"z={z}, e={e}")
        return e
    
    def add(self, x):
        """Add a node id to the counter.
        
        Parameters
        ----------
        x : int
            A node id.
        
        Returns
        -------
        None
        """
        # hash the node id
        xh = self.hash_func(x)
        # get the left self.b bit sequence and the self.hash_to - self.b sequence from the right (the rest)
        l, r = self.get_left_right_split(xh)
        # save old value
        old = self.counter[l]
        # get the number of leading zeroes + 1 of the right hand side bit sequence.
        l0s = self.get_leading_zeros_plus_one(r)
        # update counter
        self.counter[l] = max(old, l0s)
        if self.debug:
            print(f"before={old}, after={self.counter[l]}")
    
    def __repr__(self, ):
        """Repr."""
        repr_str = f"b={self.b}, p={self.p}, "

In [22]:
b = 5
hash_to = 32
debug = False
hllc1 = HyperLogLogCounter(b=b, hash_to=hash_to, debug=debug)

for xs in list(g.nodes):
    hllc1.add(xs)
    
print(hllc1.size())


b = 5
hash_to = 32
debug = False
hllc2 = HyperLogLogCounter(b=b, hash_to=hash_to, debug=debug)

for xs in list(g.edges):
    hllc2.add(xs[0])
    hllc2.add(xs[1])
    
print(hllc2.size())

l1 = [500 for i in range(1000)]
l2 = [21 for i in range(2000)]
l3 = [400 for i in range(500)]
l3 = [52 for i in range(50)]
l = l1 + l2 + l3

hllc3 = HyperLogLogCounter(b=b, hash_to=hash_to, debug=debug)

for xs in l:
    hllc3.add(xs)
    
print(hllc3.size())

6047.45450143158
6047.45450143158
0.0


In [29]:
class HyperBall():
    """ HyperBall from https://arxiv.org/pdf/1308.2144v2.pdf
    alpha is calcualted as in http://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf
    """
    def __init__(self, b, hash_to, g):
        """Init.
        
        Parameters
        ----------
        b : int
            Number of bits in counter registers. Defines the number of registers p=2^b.
        hash_to : int
            Hashing to n bits. E.g.: 32 so hasing items (str or str of int) to 32 bits integers.
        g : networkx.DiGraph
            Directed graph with inverted edges.
        """
        # set the number of bits in registers, and the number of registers in the 
        # registers of the HyperLogLogCounters
        self.b = b
        self.p = 2**self.b
        # g is with reversed edges!
        self.g = g
        # fixed node ids in list
        self.nodes = list(g.nodes)
        # init hyperloglogcounters for each node in graph
        self.counters = [HyperLogLogCounter(b=b, hash_to=hash_to) for node in g.nodes]
        # init counters with radius=0
        [self.counters[idx].add(node) for idx, node in enumerate(self.nodes)]
        # if converged
        self.converged = False
        print(f"HyperBall: {self.__repr__()}")

    def union(self, c1, c2):
        """The unnioin of two counters is the counter with the register-wise max of registers.
        
        Parameters
        ----------
        c1 : HyperLogLogCounter
            A HyperLogLogCounter
        c2 : HyperLogLogCounter
            Another HyperLogLogCounter
        
        Returns
        -------
        c_union: HyperLogLogCounter
            The unnioin of two counters is the counter with the register-wise max of registers.
        """
        c_union = copy.deepcopy(c1)
        for i in range(self.p):
            c_union.counter[i] = max(c1.counter[i], c2.counter[i])
        return c_union
    
    def __call__(self,):
        """Fit the HyperBal to estimate the harmonic centrality of each node in the directed graph self.g.reverse()
        
        Parameters
        ----------
        None
        
        Returns
        -------
        dict
            A dict with node ids as keys and their corresponding hamronic centralities as values.
        """
        print(f"Fitting HyperBall ...")
        # start from radius=1, as radius=0 is accounted for when initalizing the HyperLogLogCounters per node
        radius = 1
        
        harmonic_centrality_estimates_per_radius = []
        converged_ratio = 0.0
        
        # while not converged, do
        while not self.converged:
            self.converged = True
            counters_old = copy.deepcopy(self.counters)
            
            # tqdm
            n_edges = len(self.g.edges)
            edges_tqdm = tqdm(enumerate(self.g.edges), total=n_edges, file=sys.stdout, bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
            edges_tqdm.set_description(f"radius={radius}, convergence: {converged_ratio*100:.4f}%, edges: ")
            
            # iterate over edges in teh reversed directed graph
            for idx_edge, (node, neighbour) in edges_tqdm:
                idx_node = self.nodes.index(node)
                idx_neighbour = self.nodes.index(neighbour)
                
                # get counter for node = estimate of the set size of the ball at radius t-1
                a = self.counters[idx_node]
                # update the counter for node = estimate of the set size of the ball at radius t
                a = self.union(c1=a, c2=counters_old[idx_neighbour])
                
                # if ball sizes at radius t and t-1 are not the same, then algorithm is not converged
                if a.size() != self.counters[idx_node].size():
                    converged_ratio += 1
                    self.converged = False
                
                # update the counters = new ball sizes for node
                self.counters[idx_node] = copy.deepcopy(a)
                
            converged_ratio = abs(n_edges - converged_ratio) / n_edges
            
            # compute the harmonic centrailities
            # (self.counters[i].size() - counters_old[i].size()) is the estimate of 
            # the number of nodes at distance t from x), paper p.4, rhs, 2nd equation
            harmonic_centrality_estimates_per_radius.append(
                [(1/radius) * (self.counters[i].size() - counters_old[i].size()) for i, node in enumerate(self.nodes)])
            
            # increase radius
            radius += 1
        
        print(f"HyperBall converged")
        # reduce sum on axis=0 (per node), i.e.: marginalize over radii
        harmonic_centrality_estimates = np.array(harmonic_centrality_estimates_per_radius).sum(axis=0)
        
        # return dict: keys node ids, values harmonic centralities
        return {node:harmonic_centrality_estimates[idx] for idx, node in enumerate(self.nodes)}
                
    def __repr__(self):
        """Repr."""
        repr_str = f"b={self.b}, p={self.p}, len(g.nodes)={len(g.nodes)}, len(g.edges)={len(g.edges)}"
        return repr_str

In [30]:
start_time = time.time()
b = 5
hash_to = 64
harmonic_centrality_hb = HyperBall(b=b, hash_to=hash_to, g=g.reverse())()
exec_time = time.time() - start_time
harmonic_centrality_hb

HyperBall: b=5, p=32, len(g.nodes)=7115, len(g.edges)=103689
Fitting HyperBall ...
radius=8, convergence: 99.9913%, edges: : 100%|██████████| 103689/103689 [00:13<00:00, 7506.87it/s]                                                                                                        
HyperBall converged


{30: 1767.4761097276078,
 1412: 1638.6234818965695,
 3352: 2336.236742454911,
 5254: 2365.965862984791,
 5543: 2284.871648179348,
 7478: 2116.975856076207,
 3: 1455.6435443054354,
 28: 1839.7913489252,
 39: 1611.7722031254675,
 54: 1797.4234566886719,
 108: 1373.5229815491114,
 152: 1828.2587455532607,
 178: 1643.0145303901927,
 182: 1628.597676975751,
 214: 2200.7775605402526,
 271: 1926.5676714533377,
 286: 1505.4874250643682,
 300: 1451.2994036018315,
 348: 1573.969175241168,
 349: 1388.7554659364582,
 371: 1615.8801252318547,
 567: 1623.4288500347736,
 581: 1387.7944832483756,
 584: 1619.138321773697,
 586: 1619.5628289273477,
 590: 1643.0878957496311,
 604: 1383.8532057776372,
 611: 1555.955356066232,
 8283: 1486.528966930271,
 25: 0.0,
 6: 1530.6578466271358,
 8: 1810.3144088823337,
 19: 1418.2061235917238,
 23: 1784.672528683832,
 29: 1542.9194329445288,
 33: 1800.963285556354,
 35: 1866.1841977364859,
 50: 1580.3183718680664,
 55: 1807.3959841519256,
 75: 1603.86963426865,
 80:

In [31]:
harmonic_centrality_true = nx.harmonic_centrality(g)
harmonic_centrality_true_sorted = {k:harmonic_centrality_true[k] for k in sorted(harmonic_centrality_true)}
harmonic_centrality_true_sorted

{3: 1407.059920634978,
 4: 0,
 5: 0,
 6: 1444.474206349252,
 7: 0,
 8: 1670.5345238094908,
 9: 0,
 10: 1328.6789682540184,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 2320.326190476195,
 16: 0,
 17: 0,
 18: 0,
 19: 1341.2289682540274,
 20: 0,
 21: 0,
 22: 0,
 23: 1626.6773809523559,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 1764.784523809473,
 29: 1478.1408730159176,
 30: 1600.3607142856902,
 31: 0,
 32: 1584.7940476190267,
 33: 1663.3107142856861,
 34: 1336.8289682540274,
 35: 1759.8107142856727,
 36: 1441.62182539688,
 37: 0,
 38: 1618.0773809523569,
 39: 1531.4011904762058,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 1642.1773809523538,
 50: 1508.4238095238306,
 51: 0,
 52: 0,
 53: 0,
 54: 1659.1440476190205,
 55: 1675.927380952349,
 56: 1971.851190476093,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 1648.2773809523537,
 62: 0,
 63: 0,
 64: 1562.7678571428455,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 71: 0,
 72: 2030.4595238094257,
 73: 0,
 74: 0,
 75: 1518.0904761904937,
 76: 

In [44]:
def approx_err(dict_est, dict_true):
    """https://en.wikipedia.org/wiki/Approximation_error"""
    error = 0
    for k, v in dict_est.items():
        error += abs((v - dict_true[k]) / (dict_true[k] + 10e-9))

    return (error / len(dict_est))

def approx_err_2(estimated, true):
    """https://en.wikipedia.org/wiki/Approximation_error"""
    return abs((estimated - true) / (true + 10e-9))


The approximation error in a data value is the discrepancy between an exact value and some approximation to it.

In [33]:
harmonic_centrality_hb_sorted = {k:harmonic_centrality_hb[k] for k in sorted(harmonic_centrality_hb)}
df_hb = pd.DataFrame(harmonic_centrality_hb_sorted.items(), columns=['id', 'hb'])
df_true = pd.DataFrame(harmonic_centrality_true_sorted.items(), columns=['id', 'true'])
df_hb.drop("id", axis=1, inplace=True)
df = pd.concat([df_true, df_hb], axis=1)
# https://en.wikipedia.org/wiki/Approximation_error
df["err [%]"] = 100 * abs((df["hb"] - df["true"]) / (df["true"] + 10e-9))

print(f"HyperBall converged with an approximation error of {df["err [%]"].mean()*100:.4f}%")

path_save_to = Path("assets/email.csv") if experiment == "email" else Path("assets/wiki.csv")
df.to_csv(path_save_to)

HyperBall converged with an approximation error of 3.9103%


,id,true,hb,err [%]
0,3,1407.059921,1455.643544,3.452847
1,4,0.000000,0.000000,0.000000
2,5,0.000000,0.000000,0.000000
3,6,1444.474206,1530.657847,5.966437
4,7,0.000000,0.000000,0.000000
...,...,...,...,...
7110,8293,2047.642857,2235.904350,9.194059
7111,8294,2002.751190,2140.414990,6.873734
7112,8295,1986.109524,2170.947846,9.306552
7113,8296,1576.327381,1685.660199,6.935921


In [89]:
df["err [%]"].mean()

3.910292179775438